In [12]:
from langdetect import detect
import json

In [118]:
def normalize_rate(r):
    if r > 3:
        return 1
    elif r < 3:
        return -1
    else:
        return 0

# TODO: include adv and shortcomings
# TODO: include votes
# TODO: clean html tags from text
# TODO: handle case when text is like '*****' 
def build_corpus(f):
    X, y = [], []
    
    with open(f, 'r') as rows:
        for row in rows:
            try:
                comment = json.loads(row)

                if len(comment['text']) > 2 and comment['rate']:
                    if comment['text'] and detect(comment['text']) == 'uk':
                        X.append(comment['text'])
                        y.append(normalize_rate(comment['rate']))

            except:
                print(comment)
    return X, y

In [124]:
X, y = build_corpus('comments.json')

{'id': 15249222, 'name': 'Дмитрий', 'rate': 5, 'text': '...', 'advantages': 'качество работы, тихий, дизайн', 'shortcomings': 'возможно цена, небольшей бак для воды.', 'positive_votes_count': 1, 'negative_votes_count': 0}
{'id': 41851414, 'name': 'Гончаренко Наталья Валерьевна', 'rate': 5, 'text': '*****', 'advantages': '', 'shortcomings': '', 'positive_votes_count': 0, 'negative_votes_count': 2}
{'id': 12403794, 'name': 'Петро Прудивус', 'rate': 4, 'text': ':))', 'advantages': 'покупкою задоволений, простий в монтажі. ;))', 'shortcomings': 'відсутній шнур живлення', 'positive_votes_count': 0, 'negative_votes_count': 0}
719 719


In [130]:
from collections import Counter

stats = Counter(y)

print('Positive: ', stats[1])
print('Neutral: ', stats[0])
print('Negative: ', stats[-1])
print('Total: ', len(X))

len(X) == len(y)

Positive:  846
Neutral:  69
Negative:  118
Total:  1033


True

In [128]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1789)

In [115]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(X_train)

<740x5711 sparse matrix of type '<class 'numpy.int64'>'
	with 17441 stored elements in Compressed Sparse Row format>

In [116]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(vectorizer.transform(X_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [117]:
from sklearn.metrics import classification_report

X_features_test = vectorizer.transform(X_test)
y_pred = clf.predict(X_features_test)

target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.80      0.15      0.26        26
     Neutral       0.00      0.00      0.00        15
    Positive       0.85      1.00      0.92       206

    accuracy                           0.85       247
   macro avg       0.55      0.38      0.39       247
weighted avg       0.79      0.85      0.79       247



/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
